In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-books-reviews/books_data.csv
/kaggle/input/amazon-books-reviews/Books_rating.csv


# Exploratory Data Analysis

In [5]:
books_rating_path = "/kaggle/input/amazon-books-reviews/Books_rating.csv"
books_data_path = "/kaggle/input/amazon-books-reviews/books_data.csv"

In [8]:
book_rating = pd.read_csv(books_rating_path, header=0)
book_rating

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,NaN,NaN,NaN,14/19,4.0,937612800,Difficult,"This is an extremely difficult book to digest,..."
2999996,B000NSLVCU,The Idea of History,NaN,A1SMUB9ASL5L9Y,jafrank,1/1,4.0,1331683200,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2999997,B000NSLVCU,The Idea of History,NaN,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",0/0,4.0,1180224000,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2999998,B000NSLVCU,The Idea of History,NaN,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",1/11,5.0,1163030400,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


In [33]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [32]:
# Create a dataset, a dataloader
class CSVDataset(Dataset):
    def __init__(self, df=None, csv_path=None, header=0):
        super().__init__()
        if df is not None:
            self.df = df
        elif csv_path is not None:
            self.df = pd.read_csv(csv_path, header=header)
        # Fillna
        self.df.fillna({"User_id": "00000000000000"}, inplace=True) # Use '00000000000000' for unknown User_id
        self.df.fillna({"Id": "0000000000"}, inplace=True) # Use '0000000000' for unknown book ID
        # Book ID lookup
        raw_book_ids = sorted(list(set(self.df.Id))) # sort raw ID for reproduceability
        self.books = [raw_book_id for idx, raw_book_id in enumerate(raw_book_ids)]
        self.book2idx = {raw_book_id:idx for idx, raw_book_id in enumerate(raw_book_ids)}
        self.idx2book = {idx:raw_book_id for idx, raw_book_id in enumerate(raw_book_ids)}
        # User ID lookup
        raw_user_ids = sorted(list(set(self.df.User_id))) # sort raw ID for reproduceability
        self.users = [raw_user_id for raw_user_id in enumerate(raw_user_ids)]
        self.user2idx = {raw_user_id:idx for idx, raw_user_id in enumerate(raw_user_ids)}
        self.idx2user = {idx:raw_user_id for idx, raw_user_id in enumerate(raw_user_ids)}
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx, :]
        # Return dataset
        return {
            "user": {
                "id": self.user2idx.get(row.User_id, 0) # 0 is unknown user ID
            },
            "item": {
                "id": self.book2idx.get(row.Id, 0) # 0 is unknown book ID
            },
            "label": 1
        }

# books_rating_ds = CSVDataset(csv_path=books_rating_path)
books_rating_ds = CSVDataset(df=book_rating)

books_rating_ds[0]

{'user': {'id': 974367}, 'item': {'id': 130707}, 'label': 1}

In [37]:
# Create user tower and item tower

class UserTower(nn.Module):
    def __init__(self, num_users, id_emb_dim, tower_emb_dim):
        """
        User tower that converts user features into a user embedding for dot product.

        Args:
        num_users - total users
        id_emb_dim - Dimension of user ID embedding
        tower_emb_dim - Dimension of user tower embedding
        """
        super().__init__()
        self.id_emb = nn.Embedding(num_users, id_emb_dim)
        self.mlp = nn.Sequential(
            nn.Linear(id_emb_dim, tower_emb_dim),
            nn.ReLU(),
            nn.Linear(tower_emb_dim, tower_emb_dim)
        )

    def forward(self, user_id):
        x = self.id_emb(user_id) # [B, id_emb_dim]
        x = self.mlp(x)
        # Apply L2 normalization so to enable cosine similarity using x instead of unbounded dot product
        x = x / x.norm(dim=-1, keepdim=True)
        return x


user_id_emb_dim = 32
user_tower_emb_dim = 32
num_users = len(books_rating_ds.user2idx)

user_tower = UserTower(num_users, user_id_emb_dim, user_tower_emb_dim)
user_tower

UserTower(
  (id_emb): Embedding(1008973, 32)
  (mlp): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
)

In [ ]:
# Create two-towers model

In [ ]:
# Train / Test split

In [ ]:
# Create a in-batch negative dataset that generates negative training samples from other user's positive items
# The negative sampling dataset needs to be created for training and test individually

In [ ]:
# Train the two-towers model and monitor the loss and training epoch
# * Use Binary Cross Entropy loss
# * Switch to Negative Contrastive Ex loss

In [ ]:
# Calculate offline performance metrics on the train and test set
# Hit@K
# Recall@K
# Normalised Weighted Cumulative Gain@K
# Mean Reciprocal Rank@K